In [ ]:
# Import libraries
import os
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, JSONLoader, UnstructuredXMLLoader


import chromadb
from uuid import uuid4
from chromadb.utils import embedding_functions

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# Helper functions
def print_chunk_info(chunks):
   print(f'No of chunks: {len(chunks)}')
   idx = random.randrange(0, len(chunks))
   print(f'Chunk index: {idx}')
   print('Chunk details')
   for k, v in enumerate(chunks[idx]):
      print(f'\t{k} = {v}')

In [ ]:
# Import the questions


# Import the answers


In [ ]:
# TODO: Create a text splitter
chunk_size = 300
chunk_overlap = 30

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

In [ ]:
# Load Notes (PDF)

pdf_loader = PyPDFLoader(file_path='./docs/Path-to-GitOps-Red-Hat-Developer-e-book.pdf', mode="page", extract_images=True)
chunks = pdf_loader.load_and_split(text_splitter)

print_chunk_info(chunks)

In [ ]:
# Create embeddings

# TODO: Create embedding model
embed_model_name = "BAAI/bge-small-en-v1.5"
#embed_model_name = "all-MiniLM-L6-v2"

chroma_embed_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=embed_model_name)

# TODO: Prepare the chunks for inserting into Chroma
# Extract the text
texts = [ c.page_content for c in chunks ]
print(texts[100])
print(len(texts))

text_ids = [  str(uuid4())[:8] for _ in range(len(texts))]
print(text_ids)
print(len(text_ids))

In [ ]:
# Insert the chunks embdedings into ChromaDB

# TODO: Create ephemeral Chroma client and save chunks
col_name = 'carol'

# Create a the chromadb client
ch_client = chromadb.Client()

# drop the table
try:
   ch_client.delete_collection(col_name)
except:
   pass

# Insert the texts into the database
carol_col = ch_client.create_collection(
   name = col_name,
   embedding_function=chroma_embed_func
)

#Insert the docs into the collection
carol_col.add(
   documents = texts,
   ids = text_ids
)

In [ ]:
# RAG retrieval (Individual question)
model_name = "google/flan-t5-base"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Extract the core ideas of the question 
question = "What is the name of Scrooge's underpaid clerk?"
#question = "Who was Scrooge's deceased business partner?"
#question = "Who was Scrooge engaged to in his youth, and why did she leave him?"
question = "What is the name of Bob Cratchit's youngest son who is ill?"
#question = "What does Scrooge see written on the gravestone that frightens him into changing his ways?"
#question = " What is Scrooge's response when his nephew Fred invites him to Christmas dinner at the beginning of the story?"
question = " What specific, generous act does Scrooge perform for the Cratchit family on Christmas morning?"
question = "Who is Voldermort?"

prompt = f"{question}\n\nWhat is sentence that verbalizes this data?"
#prompt = f"{question}\n\nWhat data can be extracted from this sentence?"
#prompt = f"Generate an approximately fifteen-word sentence that describes all this data: {question}"

# convert to a statement
enc_prompt = tokenizer(prompt, return_tensors='pt')
enc_answer = model.generate(enc_prompt.input_ids)
answer = tokenizer.decode(enc_answer[0], skip_special_tokens=True)

print(answer)

In [ ]:
# TODO Your code
context = ""
results = carol_col.query(
   query_texts=[ answer ],
   n_results=3
)
print(results['distances'])
for id in results['ids'][0]:
   result = carol_col.get(id)
   context += result['documents'][0]

print(context)

In [ ]:
question_prompt = f"Answer based on context:\n\n{context}\n\n{question}"
print(question_prompt)

In [ ]:
# TODO Your code
enc_query_prompt = tokenizer(question_prompt, return_tensors='pt')

enc_query_answer = model.generate(enc_query_prompt.input_ids)

query_answer = tokenizer.decode(enc_query_answer[0], skip_special_tokens=True)

print(question)
print(query_answer)

In [ ]:
# Answer all 31 questions and calculate metrics (Base Model)





In [ ]:
# Answer all 31 questions and calculate metrics (RAG Model)



